# 01 - State-Level Data Cleaning
## Trigeminal Neuralgia Treatment Patterns Analysis

**Data Source:** Epic Cosmos  
**Study Period:** November 28, 2022 - November 27, 2025 (3 years)  
**ICD-10 Code:** G50.0 (Trigeminal Neuralgia)  
**Target Journal:** Journal of Neurosurgery (JNS)

---

### Purpose
This notebook performs data cleaning and preprocessing on state-level data from Epic Cosmos.
The goal is to prepare clean, analysis-ready datasets for:
1. Medication utilization patterns by state
2. Procedure utilization patterns by state
3. Medication-to-procedure treatment pathways

### Important Notes

**Privacy Masking:** Epic Cosmos masks cell values ≤10 as "10 or fewer" for patient privacy.  
**Imputation Decision:** We impute "10 or fewer" → 5 (midpoint of [1,10] range).  
This is a conservative estimate documented per research protocol.

---


## 1. Setup and Imports


In [26]:
# Standard library imports
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Add project root to path for imports
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Import project configuration and utilities
from src.config import (
    RAW_DATA_DIR, PROCESSED_DATA_DIR, DATA_FILES,
    SMALL_CELL_VALUE, SMALL_CELL_IMPUTATION,
    TN_CONFIG, STATE_TO_REGION, CENSUS_REGIONS,
    ensure_directories
)
from src.utils.data_cleaning import (
    impute_small_cells, clean_column_names, extract_epic_data,
    add_census_region, validate_totals, check_missing_states
)

# Ensure output directories exist
ensure_directories()

print(f"Project Root: {project_root}")
print(f"Raw Data Dir: {RAW_DATA_DIR}")
print(f"Processed Data Dir: {PROCESSED_DATA_DIR}")
print(f"\nSmall cell value: '{SMALL_CELL_VALUE}' → Imputed as: {SMALL_CELL_IMPUTATION}")


Project Root: /Users/dhirajpangal/Library/Mobile Documents/com~apple~CloudDocs/Desktop/RESEARCH/STANFORD/Cosmos/trigeminalneuralgia-cosmos
Raw Data Dir: /Users/dhirajpangal/Library/Mobile Documents/com~apple~CloudDocs/Desktop/RESEARCH/STANFORD/Cosmos/trigeminalneuralgia-cosmos/TN_Data
Processed Data Dir: /Users/dhirajpangal/Library/Mobile Documents/com~apple~CloudDocs/Desktop/RESEARCH/STANFORD/Cosmos/trigeminalneuralgia-cosmos/analysis/outputs/data

Small cell value: '10 or fewer' → Imputed as: 5


## 2. Load Raw Data Files

We have 3 state-level files:
1. **TN Medication Data.xlsx** - Medication counts by state
2. **TN procedures only.xlsx** - Procedure counts by state  
3. **TN meds and procedures.xlsx** - Cross-tabulation of medications × procedures by state


In [27]:
# Define file paths
meds_file = RAW_DATA_DIR / DATA_FILES.medications_by_state
procs_file = RAW_DATA_DIR / DATA_FILES.procedures_by_state
meds_procs_file = RAW_DATA_DIR / DATA_FILES.meds_procedures_by_state

print("Files to process:")
print(f"  1. {meds_file.name}")
print(f"  2. {procs_file.name}")
print(f"  3. {meds_procs_file.name}")


Files to process:
  1. TN Medication Data.xlsx
  2. TN procedures only.xlsx
  3. TN meds and procedures.xlsx


### 2.1 Inspect Raw File Structure

Epic Cosmos exports include metadata headers. We need to identify where the actual data begins.


In [28]:
# Inspect the medications file structure
print("=" * 80)
print("MEDICATIONS FILE - First 15 rows (raw)")
print("=" * 80)

df_meds_raw = pd.read_excel(meds_file, header=None)
print(df_meds_raw.head(15).to_string())


MEDICATIONS FILE - First 15 rows (raw)
                                                                                                                          0                                                             1                                      2                                                          3            4           5                   6                  7                                                                                     8
0                                                                                                             Session Title  Number of Patients by State of Residence and All Medications                                    NaN                                                        NaN          NaN         NaN                 NaN                NaN                                                                                   NaN
1                                                                                            

In [29]:
# Inspect the procedures file structure
print("=" * 80)
print("PROCEDURES FILE - First 15 rows (raw)")
print("=" * 80)

df_procs_raw = pd.read_excel(procs_file, header=None)
print(df_procs_raw.head(15).to_string())


PROCEDURES FILE - First 15 rows (raw)
                                                                                                                          0                                                               1                                      2                                                          3                   4                                                       5                  6                                                                                     7
0                                                                                                             Session Title  Number of Patients by State of Residence and Billed Procedures                                    NaN                                                        NaN                 NaN                                                     NaN                NaN                                                                                   NaN
1                           

## 3. Clean Medications Data (State-Level)


In [30]:
# Read medications data - headers start at row 11 (0-indexed)
# Row 11 contains: "All Medications", medication names, Total
# Row 12 contains: "State of Residence"
# Row 13+ contains: actual data

df_meds_raw = pd.read_excel(meds_file, header=None)

# Extract column headers from row 11
medication_headers = df_meds_raw.iloc[11].tolist()
print("Medication headers (row 11):")
print(medication_headers)


Medication headers (row 11):
['All Medications', 'Carbmazapine or Oxcarbmazapine', 'baclofen', 'gabapentin', 'lamotrigine', 'pregabalin', 'onabotulinumtoxinA', 'None of the above', 'Total: Total includes all data, including data from columns not currently displayed.']


In [31]:
# Data starts at row 13 (after header rows and "State of Residence" row)
# First column is state, remaining columns are medication counts

# Build proper column names
columns = ['state']
for h in medication_headers[1:]:
    h_str = str(h).strip() if pd.notna(h) else ''
    if 'Total' in h_str:
        columns.append('total')
    elif h_str and h_str not in ['nan', 'NaN']:
        columns.append(h_str)

print(f"Column names: {columns}")

# Extract data rows (row 13 to end, excluding Total row)
df_meds = df_meds_raw.iloc[13:].copy()
df_meds = df_meds.iloc[:, :len(columns)]  # Keep only relevant columns
df_meds.columns = columns
df_meds = df_meds.reset_index(drop=True)

# Display first few rows
print(f"Shape: {df_meds.shape}")
print("\nFirst 10 rows:")
df_meds.head(10)


Column names: ['state', 'Carbmazapine or Oxcarbmazapine', 'baclofen', 'gabapentin', 'lamotrigine', 'pregabalin', 'onabotulinumtoxinA', 'None of the above', 'total']
Shape: (65, 9)

First 10 rows:


,state,Carbmazapine or Oxcarbmazapine,baclofen,gabapentin,lamotrigine,pregabalin,onabotulinumtoxinA,None of the above,total
0,Ohio,9362,3819,11319,1497,3092,1371,4707,21746
1,Texas,9903,3450,11883,1199,3902,948,3596,20732
2,Florida,7573,2991,9219,930,2795,708,4202,18178
3,Pennsylvania,7411,3117,9223,1429,2580,1109,4038,18076
4,California,5840,2466,7795,764,2143,760,3823,15192
5,North Carolina,6141,2741,8072,1076,2696,703,2757,14442
6,New York,5932,2009,6907,845,1844,626,3605,14139
7,Michigan,4852,1903,5523,807,1890,601,2718,11672
8,Virginia,4458,1857,5502,753,1922,671,2818,11515
9,Illinois,5193,1946,5770,759,1861,404,2336,11300


In [32]:
# Clean the medications dataframe

# 1. Remove any rows where state is NaN or contains 'Total'
df_meds = df_meds[df_meds['state'].notna()]
df_meds = df_meds[~df_meds['state'].astype(str).str.contains('Total', na=False)]

# 2. Standardize medication column names
column_mapping = {
    'Carbmazapine or Oxcarbmazapine': 'carbamazepine_oxcarbazepine',
    'baclofen': 'baclofen',
    'gabapentin': 'gabapentin',
    'lamotrigine': 'lamotrigine',
    'pregabalin': 'pregabalin',
    'onabotulinumtoxinA': 'onabotulinumtoxina',
    'None of the above': 'none_of_above'
}

df_meds = df_meds.rename(columns=column_mapping)

# 3. Impute "10 or fewer" values
medication_cols = list(column_mapping.values()) + ['total']
medication_cols = [c for c in medication_cols if c in df_meds.columns]

df_meds = impute_small_cells(df_meds, columns=medication_cols)

# 4. Convert to numeric
for col in medication_cols:
    df_meds[col] = pd.to_numeric(df_meds[col], errors='coerce')

# 5. Add census region
df_meds = add_census_region(df_meds, state_column='state')

# 6. Filter to US states + DC only (exclude territories and international)
# This removes Puerto Rico, Virgin Islands, Canadian provinces, Mexican states, etc.
n_before = len(df_meds)
patients_before = df_meds['total'].sum()
df_meds = df_meds[df_meds['census_region'].notna()].copy()
n_after = len(df_meds)
patients_after = df_meds['total'].sum()

print(f"Filtered to US states + DC only:")
print(f"  Removed: {n_before - n_after} entities, {patients_before - patients_after:,} patients")
print(f"  Remaining: {n_after} states, {patients_after:,} patients")

print(f"\nCleaned medications data: {df_meds.shape}")
print(f"Columns: {df_meds.columns.tolist()}")
df_meds.head(10)


Filtered to US states + DC only:
  Removed: 13 entities, 2,891 patients
  Remaining: 51 states, 290,088 patients

Cleaned medications data: (51, 10)
Columns: ['state', 'carbamazepine_oxcarbazepine', 'baclofen', 'gabapentin', 'lamotrigine', 'pregabalin', 'onabotulinumtoxina', 'none_of_above', 'total', 'census_region']


,state,carbamazepine_oxcarbazepine,baclofen,gabapentin,lamotrigine,pregabalin,onabotulinumtoxina,none_of_above,total,census_region
0,Ohio,9362,3819,11319,1497,3092,1371,4707,21746,East North Central
1,Texas,9903,3450,11883,1199,3902,948,3596,20732,West South Central
2,Florida,7573,2991,9219,930,2795,708,4202,18178,South Atlantic
3,Pennsylvania,7411,3117,9223,1429,2580,1109,4038,18076,Middle Atlantic
4,California,5840,2466,7795,764,2143,760,3823,15192,Pacific
5,North Carolina,6141,2741,8072,1076,2696,703,2757,14442,South Atlantic
6,New York,5932,2009,6907,845,1844,626,3605,14139,Middle Atlantic
7,Michigan,4852,1903,5523,807,1890,601,2718,11672,East North Central
8,Virginia,4458,1857,5502,753,1922,671,2818,11515,South Atlantic
9,Illinois,5193,1946,5770,759,1861,404,2336,11300,East North Central


In [33]:
# Validate: Check for missing states
missing_states = check_missing_states(df_meds, 'state')
print(f"States in data: {len(df_meds['state'].unique())}")
print(f"Missing US states: {missing_states if missing_states else 'None'}")

# Check for NaN values
print(f"\nNaN values per column:")
print(df_meds.isna().sum())


States in data: 51
Missing US states: None

NaN values per column:
state                          0
carbamazepine_oxcarbazepine    0
baclofen                       0
gabapentin                     0
lamotrigine                    0
pregabalin                     0
onabotulinumtoxina             0
none_of_above                  0
total                          0
census_region                  0
dtype: int64


In [34]:
# Summary statistics
print("=" * 80)
print("MEDICATIONS DATA - Summary Statistics")
print("=" * 80)

# Get medication columns (excluding state, total, census_region)
med_cols = [c for c in df_meds.columns if c not in ['state', 'total', 'census_region']]

print(f"\nTotal unique states: {df_meds['state'].nunique()}")
print(f"Total patients (sum of state totals): {df_meds['total'].sum():,}")

print(f"\nMedication totals across all states:")
for col in med_cols:
    total = df_meds[col].sum()
    print(f"  {col}: {total:,}")


MEDICATIONS DATA - Summary Statistics

Total unique states: 51
Total patients (sum of state totals): 290,088

Medication totals across all states:
  carbamazepine_oxcarbazepine: 123,851
  baclofen: 47,736
  gabapentin: 148,000
  lamotrigine: 19,266
  pregabalin: 47,032
  onabotulinumtoxina: 14,972
  none_of_above: 63,719


## 4. Clean Procedures Data (State-Level)


In [35]:
# Read procedures data
df_procs_raw = pd.read_excel(procs_file, header=None)

# Headers are at row 11 (row 10 is empty)
procedure_headers = df_procs_raw.iloc[11].tolist()
print("Procedure headers (row 11):")
for i, h in enumerate(procedure_headers):
    if pd.notna(h):
        print(f"  [{i}]: {h}")


Procedure headers (row 11):
  [0]: Billed Procedures
  [1]: CRNEC SOPL EXPLORATION/DECOMPRESSION CRANIAL NRV 61458
  [2]: SRS 61796 and 98
  [3]: CREATE LESION STRTCTC PRQ NEUROLYTIC GASSERIAN 61790
  [4]: Glycerol Rhizotomy
  [5]: CHEMODNRVTJ MUSC MUSC INNERVATED FACIAL NRV UNIL 64612
  [6]: None of the above
  [7]: Total: Total includes all data, including data from columns not currently displayed.


In [36]:
# Map CPT code descriptions to clean names
procedure_name_mapping = {
    'CRNEC SOPL EXPLORATION/DECOMPRESSION CRANIAL NRV 61458': 'mvd',
    'SRS 61796 and 98': 'srs',
    'CREATE LESION STRTCTC PRQ NEUROLYTIC GASSERIAN 61790': 'rhizotomy',
    'Glycerol Rhizotomy': 'glycerol_rhizotomy',
    'CHEMODNRVTJ MUSC MUSC INNERVATED FACIAL NRV UNIL 64612': 'botox',
    'None of the above': 'none_of_above'
}

# Build column names
proc_columns = ['state']
for h in procedure_headers[1:]:
    h_str = str(h).strip() if pd.notna(h) else ''
    if h_str in procedure_name_mapping:
        proc_columns.append(procedure_name_mapping[h_str])
    elif 'Total' in h_str:
        proc_columns.append('total')
    elif h_str and h_str not in ['nan', 'NaN']:
        proc_columns.append(h_str.lower().replace(' ', '_'))

print(f"Mapped columns: {proc_columns}")


Mapped columns: ['state', 'mvd', 'srs', 'rhizotomy', 'glycerol_rhizotomy', 'botox', 'none_of_above', 'total']


In [37]:
# Extract data rows (row 13 onwards, after headers and "State of Residence" row)
df_procs = df_procs_raw.iloc[13:].copy()
df_procs = df_procs.iloc[:, :len(proc_columns)]
df_procs.columns = proc_columns
df_procs = df_procs.reset_index(drop=True)

# Remove Total rows and NaN states
df_procs = df_procs[df_procs['state'].notna()]
df_procs = df_procs[~df_procs['state'].astype(str).str.contains('Total', na=False)]

# Impute small cells
proc_cols = [c for c in proc_columns if c != 'state']
df_procs = impute_small_cells(df_procs, columns=proc_cols)

# Convert to numeric
for col in proc_cols:
    df_procs[col] = pd.to_numeric(df_procs[col], errors='coerce')

# Add census region
df_procs = add_census_region(df_procs, state_column='state')

# Filter to US states + DC only
df_procs = df_procs[df_procs['census_region'].notna()].copy()

print(f"Cleaned procedures data (US states + DC only): {df_procs.shape}")
print(f"Columns: {df_procs.columns.tolist()}")
df_procs.head(10)


Cleaned procedures data (US states + DC only): (51, 9)
Columns: ['state', 'mvd', 'srs', 'rhizotomy', 'glycerol_rhizotomy', 'botox', 'none_of_above', 'total', 'census_region']


,state,mvd,srs,rhizotomy,glycerol_rhizotomy,botox,none_of_above,total,census_region
0,Ohio,319,79,66,169,231,20960,21746,East North Central
1,Texas,406,134,50,259,170,19777,20732,West South Central
2,Florida,412,67,192,246,97,17342,18178,South Atlantic
3,Pennsylvania,267,158,23,181,124,17398,18076,Middle Atlantic
4,California,453,66,73,94,120,14435,15192,Pacific
5,North Carolina,200,58,5,56,129,14020,14442,South Atlantic
6,New York,95,29,81,232,107,13678,14139,Middle Atlantic
7,Michigan,107,13,25,47,45,11450,11672,East North Central
8,Virginia,78,49,5,77,80,11241,11515,South Atlantic
9,Illinois,110,20,17,127,62,10989,11300,East North Central


In [38]:
# Summary statistics for procedures
print("=" * 80)
print("PROCEDURES DATA - Summary Statistics")
print("=" * 80)

proc_only_cols = [c for c in df_procs.columns if c not in ['state', 'total', 'census_region']]

print(f"\nTotal unique states: {df_procs['state'].nunique()}")
print(f"Total patients (sum of state totals): {df_procs['total'].sum():,}")

print(f"\nProcedure totals across all states:")
for col in proc_only_cols:
    total = df_procs[col].sum()
    print(f"  {col}: {total:,}")


PROCEDURES DATA - Summary Statistics

Total unique states: 51
Total patients (sum of state totals): 290,088

Procedure totals across all states:
  mvd: 4,033
  srs: 1,813
  rhizotomy: 972
  glycerol_rhizotomy: 2,616
  botox: 2,069
  none_of_above: 279,762


## 5. Clean Medications × Procedures Cross-Tabulation Data


In [39]:
# This file has medications × procedures × states (most detailed)
# Structure: State | Medication | Procedure1 | Procedure2 | ... | Total

df_cross_raw = pd.read_excel(meds_procs_file, header=None)

print("Cross-tabulation file structure (rows 8-14):")
print(df_cross_raw.iloc[8:15].to_string())


Cross-tabulation file structure (rows 8-14):
                     0                               1                                                       2                 3                                                     4                   5                                                       6                  7                                                                                     8
8       Date of Export                      12/29/2025                                                     NaN               NaN                                                   NaN                 NaN                                                     NaN                NaN                                                                                   NaN
9                  NaN                             NaN                                                     NaN               NaN                                                   NaN                 NaN                         

In [40]:
# Extract headers - Row 11 has procedure names (row 10 is empty)
# Row 11: [nan, 'Billed Procedures', procedure1, procedure2, ...]
# Row 12: ['State of Residence', 'All Medications', nan, ...]
# Row 13+: [state, medication, count1, count2, ...]

proc_header_row = df_cross_raw.iloc[11].tolist()

print("Row 11 (procedure headers):")
for i, h in enumerate(proc_header_row[:10]):
    if pd.notna(h):
        print(f"  [{i}]: {h}")


Row 11 (procedure headers):
  [1]: Billed Procedures
  [2]: CRNEC SOPL EXPLORATION/DECOMPRESSION CRANIAL NRV 61458
  [3]: SRS 61796 and 98
  [4]: CREATE LESION STRTCTC PRQ NEUROLYTIC GASSERIAN 61790
  [5]: Glycerol Rhizotomy
  [6]: CHEMODNRVTJ MUSC MUSC INNERVATED FACIAL NRV UNIL 64612
  [7]: None of the above
  [8]: Total: Total includes all data, including data from columns not currently displayed.


In [41]:
# Build column names for cross-tab data
# Columns: state | medication | mvd | srs | rhizotomy | glycerol | botox | none | total

# Get procedure headers from row 11, starting at column 2
procedure_headers = proc_header_row[2:]

cross_columns = ['state', 'medication']

# Map procedure columns
for h in procedure_headers:
    h_str = str(h).strip() if pd.notna(h) else ''
    if h_str in procedure_name_mapping:
        cross_columns.append(procedure_name_mapping[h_str])
    elif 'Total' in h_str:
        cross_columns.append('total')
    elif h_str and h_str not in ['nan', 'NaN']:
        cross_columns.append(h_str.lower().replace(' ', '_'))

print(f"Cross-tab columns ({len(cross_columns)}): {cross_columns}")


Cross-tab columns (9): ['state', 'medication', 'mvd', 'srs', 'rhizotomy', 'glycerol_rhizotomy', 'botox', 'none_of_above', 'total']


In [42]:
# Extract data (starts at row 12)
df_cross = df_cross_raw.iloc[12:].copy()
df_cross = df_cross.iloc[:, :len(cross_columns)]
df_cross.columns = cross_columns
df_cross = df_cross.reset_index(drop=True)

# Forward-fill state names (Epic merges cells for same state)
df_cross['state'] = df_cross['state'].ffill()

# Remove Total rows
df_cross = df_cross[~df_cross['state'].astype(str).str.contains('Total', na=False)]
df_cross = df_cross[~df_cross['medication'].astype(str).str.contains('Total', na=False)]
df_cross = df_cross[df_cross['medication'].notna()]

print(f"Shape before cleaning: {df_cross.shape}")
df_cross.head(20)


Shape before cleaning: (449, 9)


,state,medication,mvd,srs,rhizotomy,glycerol_rhizotomy,botox,none_of_above,total
0,State of Residence,All Medications,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ohio,Carbmazapine or Oxcarbmazapine,270,56,48,110,117,8822,9362
2,Ohio,baclofen,153,29,31,69,95,3477,3819
3,Ohio,gabapentin,211,61,48,124,144,10789,11319
4,Ohio,lamotrigine,52,17,12,29,36,1368,1497
5,Ohio,pregabalin,75,20,17,51,68,2887,3092
6,Ohio,onabotulinumtoxinA,32,10 or fewer,15,36,214,1088,1371
7,Ohio,None of the above,10 or fewer,10 or fewer,10 or fewer,12,10 or fewer,4678,4707
9,Texas,Carbmazapine or Oxcarbmazapine,339,105,46,211,85,9176,9903
10,Texas,baclofen,132,43,21,105,49,3131,3450


In [43]:
# Standardize medication names
med_name_mapping = {
    'Carbmazapine or Oxcarbmazapine': 'Carbamazepine/Oxcarbazepine',
    'baclofen': 'Baclofen',
    'gabapentin': 'Gabapentin',
    'lamotrigine': 'Lamotrigine',
    'pregabalin': 'Pregabalin',
    'onabotulinumtoxinA': 'OnabotulinumtoxinA',
    'None of the above': 'None of the above'
}

df_cross['medication'] = df_cross['medication'].replace(med_name_mapping)

# Impute small cells
numeric_cols = [c for c in cross_columns if c not in ['state', 'medication']]
df_cross = impute_small_cells(df_cross, columns=numeric_cols)

# Convert to numeric
for col in numeric_cols:
    df_cross[col] = pd.to_numeric(df_cross[col], errors='coerce')

# Add census region
df_cross = add_census_region(df_cross, state_column='state')

# Filter to US states + DC only
df_cross = df_cross[df_cross['census_region'].notna()].copy()

print(f"Cleaned cross-tab data (US states + DC only): {df_cross.shape}")
print(f"Unique states: {df_cross['state'].nunique()}")
print(f"Unique medications: {df_cross['medication'].unique().tolist()}")
df_cross.head(15)


Cleaned cross-tab data (US states + DC only): (357, 10)
Unique states: 51
Unique medications: ['Carbamazepine/Oxcarbazepine', 'Baclofen', 'Gabapentin', 'Lamotrigine', 'Pregabalin', 'OnabotulinumtoxinA', 'None of the above']


,state,medication,mvd,srs,rhizotomy,glycerol_rhizotomy,botox,none_of_above,total,census_region
1,Ohio,Carbamazepine/Oxcarbazepine,270.0,56.0,48.0,110.0,117.0,8822.0,9362.0,East North Central
2,Ohio,Baclofen,153.0,29.0,31.0,69.0,95.0,3477.0,3819.0,East North Central
3,Ohio,Gabapentin,211.0,61.0,48.0,124.0,144.0,10789.0,11319.0,East North Central
4,Ohio,Lamotrigine,52.0,17.0,12.0,29.0,36.0,1368.0,1497.0,East North Central
5,Ohio,Pregabalin,75.0,20.0,17.0,51.0,68.0,2887.0,3092.0,East North Central
6,Ohio,OnabotulinumtoxinA,32.0,5.0,15.0,36.0,214.0,1088.0,1371.0,East North Central
7,Ohio,None of the above,5.0,5.0,5.0,12.0,5.0,4678.0,4707.0,East North Central
9,Texas,Carbamazepine/Oxcarbazepine,339.0,105.0,46.0,211.0,85.0,9176.0,9903.0,West South Central
10,Texas,Baclofen,132.0,43.0,21.0,105.0,49.0,3131.0,3450.0,West South Central
11,Texas,Gabapentin,290.0,98.0,31.0,186.0,111.0,11217.0,11883.0,West South Central


## 6. Data Validation


In [44]:
# Cross-check totals between datasets
print("=" * 80)
print("DATA VALIDATION - Cross-Dataset Comparison")
print("=" * 80)

# Total patients from each dataset
meds_total = df_meds['total'].sum()
procs_total = df_procs['total'].sum()
cross_total = df_cross.groupby('state')['total'].first().sum()  # Sum unique state totals

print(f"\nTotal patients:")
print(f"  From medications data: {meds_total:,}")
print(f"  From procedures data:  {procs_total:,}")
print(f"  From cross-tab data:   {cross_total:,}")

# Note: These should be approximately equal (small differences due to imputation)
if abs(meds_total - procs_total) / procs_total < 0.01:
    print("\n✓ Totals match across datasets (within 1%)")
else:
    print(f"\n⚠ Totals differ by {abs(meds_total - procs_total):,} ({abs(meds_total - procs_total)/procs_total*100:.2f}%)")


DATA VALIDATION - Cross-Dataset Comparison

Total patients:
  From medications data: 290,088
  From procedures data:  290,088
  From cross-tab data:   123,851.0

✓ Totals match across datasets (within 1%)


In [45]:
# Check for data quality issues
print("\n" + "=" * 80)
print("DATA QUALITY CHECK")
print("=" * 80)

datasets = {
    'medications': df_meds,
    'procedures': df_procs,
    'cross_tab': df_cross
}

for name, df in datasets.items():
    print(f"\n{name.upper()}:")
    print(f"  Shape: {df.shape}")
    print(f"  NaN count: {df.isna().sum().sum()}")
    print(f"  Duplicate rows: {df.duplicated().sum()}")
    
    # Check for negative values (shouldn't exist)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    neg_values = (df[numeric_cols] < 0).sum().sum()
    print(f"  Negative values: {neg_values}")



DATA QUALITY CHECK

MEDICATIONS:
  Shape: (51, 10)
  NaN count: 0
  Duplicate rows: 0
  Negative values: 0

PROCEDURES:
  Shape: (51, 9)
  NaN count: 0
  Duplicate rows: 0
  Negative values: 0

CROSS_TAB:
  Shape: (357, 10)
  NaN count: 0
  Duplicate rows: 0
  Negative values: 0


## 7. Create Summary Tables for Quick Reference


In [46]:
# Summary: Patients by Census Region
print("=" * 80)
print("PATIENTS BY CENSUS REGION")
print("=" * 80)

region_summary = df_meds.groupby('census_region').agg({
    'state': 'count',
    'total': 'sum'
}).rename(columns={'state': 'n_states', 'total': 'n_patients'})

region_summary['pct_patients'] = (region_summary['n_patients'] / region_summary['n_patients'].sum() * 100).round(1)
region_summary = region_summary.sort_values('n_patients', ascending=False)

print(region_summary.to_string())
print(f"\nTotal: {region_summary['n_patients'].sum():,} patients")


PATIENTS BY CENSUS REGION
                    n_states  n_patients  pct_patients
census_region                                         
South Atlantic             9       65578          22.6
East North Central         5       58542          20.2
Middle Atlantic            3       37997          13.1
West South Central         4       31208          10.8
Pacific                    5       25353           8.7
West North Central         7       19768           6.8
New England                6       19196           6.6
Mountain                   8       17432           6.0
East South Central         4       15014           5.2

Total: 290,088 patients


In [47]:
# Summary: Overall Medication Utilization
print("=" * 80)
print("OVERALL MEDICATION UTILIZATION")
print("=" * 80)

med_cols = ['carbamazepine_oxcarbazepine', 'gabapentin', 'pregabalin', 
            'baclofen', 'lamotrigine', 'onabotulinumtoxina', 'none_of_above']
med_cols = [c for c in med_cols if c in df_meds.columns]

total_patients = df_meds['total'].sum()

med_summary = pd.DataFrame({
    'medication': med_cols,
    'n_patients': [df_meds[c].sum() for c in med_cols]
})
med_summary['pct'] = (med_summary['n_patients'] / total_patients * 100).round(1)
med_summary = med_summary.sort_values('n_patients', ascending=False)

print(med_summary.to_string(index=False))


OVERALL MEDICATION UTILIZATION
                 medication  n_patients  pct
                 gabapentin      148000 51.0
carbamazepine_oxcarbazepine      123851 42.7
              none_of_above       63719 22.0
                   baclofen       47736 16.5
                 pregabalin       47032 16.2
                lamotrigine       19266  6.6
         onabotulinumtoxina       14972  5.2


In [48]:
# Summary: Overall Procedure Utilization
print("=" * 80)
print("OVERALL PROCEDURE UTILIZATION")
print("=" * 80)

proc_cols = ['mvd', 'srs', 'rhizotomy', 'glycerol_rhizotomy', 'botox', 'none_of_above']
proc_cols = [c for c in proc_cols if c in df_procs.columns]

total_patients_procs = df_procs['total'].sum()

proc_summary = pd.DataFrame({
    'procedure': proc_cols,
    'n_patients': [df_procs[c].sum() for c in proc_cols]
})
proc_summary['pct'] = (proc_summary['n_patients'] / total_patients_procs * 100).round(1)
proc_summary = proc_summary.sort_values('n_patients', ascending=False)

print(proc_summary.to_string(index=False))


OVERALL PROCEDURE UTILIZATION
         procedure  n_patients  pct
     none_of_above      279762 96.4
               mvd        4033  1.4
glycerol_rhizotomy        2616  0.9
             botox        2069  0.7
               srs        1813  0.6
         rhizotomy         972  0.3


## 8. Save Cleaned Data


In [49]:
# Save cleaned datasets to CSV for downstream analysis
print("Saving cleaned datasets...")

# State-level medications
meds_output = PROCESSED_DATA_DIR / 'state_medications_clean.csv'
df_meds.to_csv(meds_output, index=False)
print(f"  ✓ Saved: {meds_output.name}")

# State-level procedures
procs_output = PROCESSED_DATA_DIR / 'state_procedures_clean.csv'
df_procs.to_csv(procs_output, index=False)
print(f"  ✓ Saved: {procs_output.name}")

# Cross-tabulation (medications × procedures × state)
cross_output = PROCESSED_DATA_DIR / 'state_meds_procedures_clean.csv'
df_cross.to_csv(cross_output, index=False)
print(f"  ✓ Saved: {cross_output.name}")

print(f"\nAll files saved to: {PROCESSED_DATA_DIR}")


Saving cleaned datasets...
  ✓ Saved: state_medications_clean.csv
  ✓ Saved: state_procedures_clean.csv
  ✓ Saved: state_meds_procedures_clean.csv

All files saved to: /Users/dhirajpangal/Library/Mobile Documents/com~apple~CloudDocs/Desktop/RESEARCH/STANFORD/Cosmos/trigeminalneuralgia-cosmos/analysis/outputs/data


## 9. Data Dictionary

### Cleaned Datasets

| File | Description | Key Columns |
|------|-------------|-------------|
| `state_medications_clean.csv` | Medication counts by US state | state, census_region, carbamazepine_oxcarbazepine, gabapentin, ... |
| `state_procedures_clean.csv` | Procedure counts by US state | state, census_region, mvd, srs, rhizotomy, ... |
| `state_meds_procedures_clean.csv` | Medication × Procedure by state | state, medication, mvd, srs, rhizotomy, ..., total |

### Column Definitions

**Medications:**
- `carbamazepine_oxcarbazepine`: First-line anticonvulsants (gold standard for TN)
- `gabapentin`: Second-line anticonvulsant
- `pregabalin`: Second-line anticonvulsant
- `baclofen`: Muscle relaxant, often used adjunctively
- `lamotrigine`: Alternative anticonvulsant
- `onabotulinumtoxina`: Botulinum toxin injection
- `none_of_above`: No documented TN-specific medication

**Procedures:**
- `mvd`: Microvascular Decompression (CPT 61458) - gold standard surgical treatment
- `srs`: Stereotactic Radiosurgery (CPT 61796) - Gamma Knife/CyberKnife
- `rhizotomy`: Percutaneous Rhizotomy (CPT 61790) - ablative procedure
- `glycerol_rhizotomy`: Glycerol injection rhizotomy
- `botox`: Botulinum toxin injection (CPT 64612)
- `none_of_above`: No documented TN-specific procedure

### Data Processing Notes

1. **Imputation**: Values reported as "10 or fewer" were imputed as 5 (midpoint)
2. **Census Regions**: States mapped to 9 US Census Bureau regions
3. **Excluded**: Non-US territories with insufficient data (<10 patients per category)


In [50]:
# Final summary
print("=" * 80)
print("DATA CLEANING COMPLETE")
print("=" * 80)
print(f"""
Study: Trigeminal Neuralgia Treatment Patterns
Data Source: Epic Cosmos
Study Period: {TN_CONFIG.study_start} to {TN_CONFIG.study_end}
ICD-10: {TN_CONFIG.icd10_code}

Datasets Created:
  1. state_medications_clean.csv ({df_meds.shape[0]} states, {df_meds.shape[1]} columns)
  2. state_procedures_clean.csv ({df_procs.shape[0]} states, {df_procs.shape[1]} columns)
  3. state_meds_procedures_clean.csv ({df_cross.shape[0]} rows, {df_cross.shape[1]} columns)

Total Patients: ~{total_patients:,}

Next Steps:
  - Run 02_descriptive_analysis_state.ipynb for descriptive statistics
  - Run 03_statistical_analysis_state.ipynb for inferential statistics
  - Run 04_tables_figures_state.ipynb for JNS-formatted outputs
""")


DATA CLEANING COMPLETE

Study: Trigeminal Neuralgia Treatment Patterns
Data Source: Epic Cosmos
Study Period: 2022-11-28 to 2025-11-27
ICD-10: G50.0

Datasets Created:
  1. state_medications_clean.csv (51 states, 10 columns)
  2. state_procedures_clean.csv (51 states, 9 columns)
  3. state_meds_procedures_clean.csv (357 rows, 10 columns)

Total Patients: ~290,088

Next Steps:
  - Run 02_descriptive_analysis_state.ipynb for descriptive statistics
  - Run 03_statistical_analysis_state.ipynb for inferential statistics
  - Run 04_tables_figures_state.ipynb for JNS-formatted outputs

